# Aggregate Summary of Statistics and Make a Baseline Table

This notebook aggregates summary of statistics of all infection-related hospital admission data and processes them to make an aggregated baseline table.

In [31]:
import pandas as pd
import numpy as np

## 0- functions and read data

In [32]:
summary_table_lrti_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_lrti/summary_table_lrti_no_abs_incdt.csv')
summary_table_lrti_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_lrti/summary_table_lrti_abs_incdt.csv')
summary_table_lrti_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_lrti/summary_table_lrti_no_abs_prevl.csv')
summary_table_lrti_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_lrti/summary_table_lrti_abs_prevl.csv')
# summary_table_urti_combined_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_urti_combined/summary_table_urti_combined_no_abs_incdt.csv')
# summary_table_urti_combined_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_urti_combined/summary_table_urti_combined_abs_incdt.csv')
# summary_table_urti_combined_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_urti_combined/summary_table_urti_combined_no_abs_prevl.csv')
# summary_table_urti_combined_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_urti_combined/summary_table_urti_combined_abs_prevl.csv')
summary_table_uti_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_uti/summary_table_uti_no_abs_incdt.csv')
summary_table_uti_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_uti/summary_table_uti_abs_incdt.csv')
summary_table_uti_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_uti/summary_table_uti_no_abs_prevl.csv')
summary_table_uti_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_uti/summary_table_uti_abs_prevl.csv')
summary_table_sinusitis_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_sinusitis/summary_table_sinusitis_no_abs_incdt.csv')
summary_table_sinusitis_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_sinusitis/summary_table_sinusitis_abs_incdt.csv')
summary_table_sinusitis_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_sinusitis/summary_table_sinusitis_no_abs_prevl.csv')
summary_table_sinusitis_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_sinusitis/summary_table_sinusitis_abs_prevl.csv')
summary_table_ot_externa_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_ot_externa/summary_table_ot_externa_no_abs_incdt.csv')
summary_table_ot_externa_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_ot_externa/summary_table_ot_externa_abs_incdt.csv')
summary_table_ot_externa_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_ot_externa/summary_table_ot_externa_no_abs_prevl.csv')
summary_table_ot_externa_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_ot_externa/summary_table_ot_externa_abs_prevl.csv')
summary_table_otmedia_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_otmedia/summary_table_otmedia_no_abs_incdt.csv')
summary_table_otmedia_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_otmedia/summary_table_otmedia_abs_incdt.csv')
summary_table_otmedia_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_otmedia/summary_table_otmedia_no_abs_prevl.csv')
summary_table_otmedia_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_otmedia/summary_table_otmedia_abs_prevl.csv')
summary_table_urti_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_urti/summary_table_urti_no_abs_incdt.csv')
summary_table_urti_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_urti/summary_table_urti_abs_incdt.csv')
summary_table_urti_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_urti/summary_table_urti_no_abs_prevl.csv')
summary_table_urti_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_urti/summary_table_urti_abs_prevl.csv')
summary_table_cough_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_cough/summary_table_cough_no_abs_incdt.csv')
summary_table_cough_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_cough/summary_table_cough_abs_incdt.csv')
summary_table_cough_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_cough/summary_table_cough_no_abs_prevl.csv')
summary_table_cough_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_cough/summary_table_cough_abs_prevl.csv')
summary_table_cough_cold_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_cough_cold/summary_table_cough_cold_no_abs_incdt.csv')
summary_table_cough_cold_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_cough_cold/summary_table_cough_cold_abs_incdt.csv')
summary_table_cough_cold_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_cough_cold/summary_table_cough_cold_no_abs_prevl.csv')
summary_table_cough_cold_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_cough_cold/summary_table_cough_cold_abs_prevl.csv')
summary_table_throat_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_throat/summary_table_throat_no_abs_incdt.csv')
summary_table_throat_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_throat/summary_table_throat_abs_incdt.csv')
summary_table_throat_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_throat/summary_table_throat_no_abs_prevl.csv')
summary_table_throat_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_throat/summary_table_throat_abs_prevl.csv')

In [33]:
def preprocess_urti_comb(data_cough,data_cough_cold,data_throat,data_urti):

    d = {'variable': data_cough['variable'],
         'count': data_cough['count'] + data_cough_cold['count'] + data_throat['count'] + data_urti['count'],
         'mean': data_cough['mean'] + data_cough_cold['mean'] + data_throat['mean'] + data_urti['mean'],
         'std': data_cough['std'] + data_cough_cold['std'] + data_throat['std'] + data_urti['std']
        }

    data_urti_comb = pd.DataFrame(data=d)

    total_cough = data_cough.iloc[data_cough[data_cough['variable']=='sex_female'].index[0],data_cough.columns.get_loc('count')] + data_cough.iloc[data_cough[data_cough['variable']=='sex_male'].index[0],data_cough.columns.get_loc('count')]
    total_cough_cold = data_cough_cold.iloc[data_cough_cold[data_cough_cold['variable']=='sex_female'].index[0],data_cough_cold.columns.get_loc('count')] + data_cough_cold.iloc[data_cough_cold[data_cough_cold['variable']=='sex_male'].index[0],data_cough_cold.columns.get_loc('count')]
    total_throat = data_throat.iloc[data_throat[data_throat['variable']=='sex_female'].index[0],data_throat.columns.get_loc('count')] + data_throat.iloc[data_throat[data_throat['variable']=='sex_male'].index[0],data_throat.columns.get_loc('count')]
    total_urti = data_urti.iloc[data_urti[data_urti['variable']=='sex_female'].index[0],data_urti.columns.get_loc('count')] + data_urti.iloc[data_urti[data_urti['variable']=='sex_male'].index[0],data_urti.columns.get_loc('count')]

    #calculate mean columns
    data_urti_comb['mean'] = ((data_cough['mean']*total_cough) + (data_cough_cold['mean']*total_cough_cold) + (data_throat['mean']*total_throat) + (data_urti['mean']*total_urti))/(total_cough + total_cough_cold + total_throat + total_urti)

    ##calculate mean and std of antibacterial_brit
    mean_cough = data_cough.iloc[data_cough[data_cough['variable']=='antibacterial_brit'].index[0],data_cough.columns.get_loc('mean')]
    mean_cough_cold = data_cough_cold.iloc[data_cough_cold[data_cough_cold['variable']=='antibacterial_brit'].index[0],data_cough_cold.columns.get_loc('mean')]
    mean_throat = data_throat.iloc[data_throat[data_throat['variable']=='antibacterial_brit'].index[0],data_throat.columns.get_loc('mean')]
    mean_urti = data_urti.iloc[data_urti[data_urti['variable']=='antibacterial_brit'].index[0],data_urti.columns.get_loc('mean')]

    std_cough = data_cough.iloc[data_cough[data_cough['variable']=='antibacterial_brit'].index[0],data_cough.columns.get_loc('std')]
    std_cough_cold = data_cough_cold.iloc[data_cough_cold[data_cough_cold['variable']=='antibacterial_brit'].index[0],data_cough_cold.columns.get_loc('std')]
    std_throat = data_throat.iloc[data_throat[data_throat['variable']=='antibacterial_brit'].index[0],data_throat.columns.get_loc('std')]
    std_urti = data_urti.iloc[data_urti[data_urti['variable']=='antibacterial_brit'].index[0],data_urti.columns.get_loc('std')]

    mean_urti_comb_antibacterial_brit = ((mean_cough*total_cough) + (mean_cough_cold*total_cough_cold) + (mean_throat*total_throat) + (mean_urti*total_urti))/(total_cough + total_cough_cold + total_throat + total_urti)
    std_urti_comb_antibacterial_brit = np.sqrt(((std_cough**2)/mean_cough) + ((std_cough_cold**2)/mean_cough_cold) + ((std_throat**2)/mean_throat) + ((std_urti**2)/mean_urti))

    data_urti_comb.iloc[data_urti_comb[data_urti_comb['variable']=='antibacterial_brit'].index[0], data_urti_comb.columns.get_loc('mean')] = mean_urti_comb_antibacterial_brit
    data_urti_comb.iloc[data_urti_comb[data_urti_comb['variable']=='antibacterial_brit'].index[0], data_urti_comb.columns.get_loc('std')] = std_urti_comb_antibacterial_brit

    #drop any row with nan
    data_urti_comb = data_urti_comb.dropna()
    
    return data_urti_comb

In [34]:
#calculating stats info of urti_combined using cough, cold with cough, sore throat, and specific urti 
summary_table_urti_combined_no_abs_incdt = preprocess_urti_comb(summary_table_cough_no_abs_incdt, summary_table_cough_cold_no_abs_incdt, summary_table_throat_no_abs_incdt, summary_table_urti_no_abs_incdt)
summary_table_urti_combined_abs_incdt = preprocess_urti_comb(summary_table_cough_abs_incdt, summary_table_cough_cold_abs_incdt, summary_table_throat_abs_incdt, summary_table_urti_abs_incdt)
summary_table_urti_combined_no_abs_prevl = preprocess_urti_comb(summary_table_cough_no_abs_prevl, summary_table_cough_cold_no_abs_prevl, summary_table_throat_no_abs_prevl, summary_table_urti_no_abs_prevl)
summary_table_urti_combined_abs_prevl = preprocess_urti_comb(summary_table_cough_abs_prevl, summary_table_cough_cold_abs_prevl, summary_table_throat_abs_prevl, summary_table_urti_abs_prevl)

In [45]:
def data_process(data):
    #replace SMs with 5 in data
    for row in range(0, len(data)):
        if data['count'][row] == 'SM':
            data['count'][row] = 5
    #conver counts into int
    data['count_int'] = data['count'].astype(int)
    #convert means into percentages with two decimals
    data['mean_perc'] = data['mean']*100
    data['mean_perc'] = np.round(data['mean_perc'], decimals=2)
    #custom order of variables to fit tables of the manuscript
    data['variable'] = pd.Categorical(data['variable'], ['age_cat_15_24', 'age_cat_25_34', 'age_cat_35_44', 'age_cat_45_54', 'age_cat_55_64', 'age_cat_65_74', 'age_cat_75_more', 'sex_female', 'sex_male', 'CCI_cat_high', 'CCI_cat_low', 'CCI_cat_medium', 'CCI_cat_very_high', 'CCI_cat_very_low', 'flu_vaccine_no', 'flu_vaccine_yes', 'bmi_cat_healthy_weight', 'bmi_cat_obese', 'bmi_cat_overweight', 'bmi_cat_underweight', 'bmi_cat_unknown', 'region_east', 'region_east_midlands', 'region_london', 'region_north_east', 'region_north_west', 'region_south_east', 'region_south_west', 'region_west_midlands', 'region_yorkshire', 'imd_affluent', 'imd_medium', 'imd_unaffluent', 'imd_unknown', 'imd_very_affluent', 'imd_very_unaffluent', 'ethnicity_white', 'ethnicity_asian', 'ethnicity_black', 'ethnicity_mixed', 'ethnicity_other', 'ethnicity_unknown', 'smoking_ex_smoker', 'smoking_never_smoked', 'smoking_smoker', 'smoking_unknown', 'season_autumn', 'season_spring', 'season_summer', 'season_winter', 'period_during_pandemic', 'period_post_2nd_lockdown', 'period_prepandemic', 'antibacterial_brit'])
    data = data.sort_values('variable')
    #keep mean values
    data['mean'] = np.round(data['mean'], decimals=2)
    #reset index to prevent issue in calculating total value
    data = data.reset_index(drop=True)
    #calculate total from sex_male and sex_female
    total = data.iloc[data[data['variable']=='sex_female'].index[0],data.columns.get_loc('count')] + data.iloc[data[data['variable']=='sex_male'].index[0],data.columns.get_loc('count')]
    #create a new dataframe for the new row and add comma to the total's thousands
    new_row = pd.DataFrame({'variable':'Total', 'count':'-', 'mean':'-', 'std':'-', 'count_int':'-', 'mean_perc':'-', 'count_mean':total}, index=[0])
    new_row['count_mean'] = new_row['count_mean'].astype(int)
    new_row['count_mean'] = new_row['count_mean'].apply(lambda x : "{:,}".format(x))
    #keep 2 digits of mean_perc
    data['mean_perc'] = data['mean_perc'].apply(lambda x: "{:.2f}".format(x))
    #add comma after 3 digits
    data['count_int'] = data['count_int'].astype(int)
    data['count_int'] = data['count_int'].apply(lambda x : "{:,}".format(x))
    #replace SN
    for row in range(0, len(data)):
        if data['count'][row] == 5:
            data['count_int'][row] = 'SN'
            data['mean_perc'][row] = 'SN'
    #combine counts and means
    data['count_mean'] = data['count_int'].astype(str) + ' (' + data['mean_perc'].astype(str) + ')'
    #append row to the dataframe
    data = pd.concat([data, new_row])
    #replace
    data['std_round'] = pd.to_numeric(data['std'],errors='coerce')
    data['std_round'] = np.round(data['std_round'], decimals=2)
    data.iloc[int(data[data['variable']=='antibacterial_brit'].index[0]),int(data.columns.get_loc('count_mean'))] = str(data.iloc[int(data[data['variable']=='antibacterial_brit'].index[0]),int(data.columns.get_loc('mean'))]) + ' (' + str(data.iloc[int(data[data['variable']=='antibacterial_brit'].index[0]),int(data.columns.get_loc('std_round'))]) + ')'
    #drop any row with nan in rows of variable
    data = data[data['variable'].notna()]
    #drop the row with period_nan in variables  
    data = data[data.variable != 'period_nan']
    #reset index
    data = data.reset_index(drop=True)
    
    return data

## 1- process data to make basline table, and save it

In [46]:
summary_table_processed_lrti_no_abs_incdt = data_process(summary_table_lrti_no_abs_incdt)
summary_table_processed_lrti_abs_incdt = data_process(summary_table_lrti_abs_incdt)
summary_table_processed_lrti_no_abs_prevl = data_process(summary_table_lrti_no_abs_prevl)
summary_table_processed_lrti_abs_prevl = data_process(summary_table_lrti_abs_prevl)
summary_table_processed_urti_combined_no_abs_incdt = data_process(summary_table_urti_combined_no_abs_incdt)
summary_table_processed_urti_combined_abs_incdt = data_process(summary_table_urti_combined_abs_incdt)
summary_table_processed_urti_combined_no_abs_prevl = data_process(summary_table_urti_combined_no_abs_prevl)
summary_table_processed_urti_combined_abs_prevl = data_process(summary_table_urti_combined_abs_prevl)
summary_table_processed_uti_no_abs_incdt = data_process(summary_table_uti_no_abs_incdt)
summary_table_processed_uti_abs_incdt = data_process(summary_table_uti_abs_incdt)
summary_table_processed_uti_no_abs_prevl = data_process(summary_table_uti_no_abs_prevl)
summary_table_processed_uti_abs_prevl = data_process(summary_table_uti_abs_prevl)
summary_table_processed_sinusitis_no_abs_incdt = data_process(summary_table_sinusitis_no_abs_incdt)
summary_table_processed_sinusitis_abs_incdt = data_process(summary_table_sinusitis_abs_incdt)
summary_table_processed_sinusitis_no_abs_prevl = data_process(summary_table_sinusitis_no_abs_prevl)
summary_table_processed_sinusitis_abs_prevl = data_process(summary_table_sinusitis_abs_prevl)
summary_table_processed_ot_externa_no_abs_incdt = data_process(summary_table_ot_externa_no_abs_incdt)
summary_table_processed_ot_externa_abs_incdt = data_process(summary_table_ot_externa_abs_incdt)
summary_table_processed_ot_externa_no_abs_prevl = data_process(summary_table_ot_externa_no_abs_prevl)
summary_table_processed_ot_externa_abs_prevl = data_process(summary_table_ot_externa_abs_prevl)
summary_table_processed_otmedia_no_abs_incdt = data_process(summary_table_otmedia_no_abs_incdt)
summary_table_processed_otmedia_abs_incdt = data_process(summary_table_otmedia_abs_incdt)
summary_table_processed_otmedia_no_abs_prevl = data_process(summary_table_otmedia_no_abs_prevl)
summary_table_processed_otmedia_abs_prevl = data_process(summary_table_otmedia_abs_prevl)
summary_table_processed_urti_no_abs_incdt = data_process(summary_table_urti_no_abs_incdt)
summary_table_processed_urti_abs_incdt = data_process(summary_table_urti_abs_incdt)
summary_table_processed_urti_no_abs_prevl = data_process(summary_table_urti_no_abs_prevl)
summary_table_processed_urti_abs_prevl = data_process(summary_table_urti_abs_prevl)
summary_table_processed_cough_no_abs_incdt = data_process(summary_table_cough_no_abs_incdt)
summary_table_processed_cough_abs_incdt = data_process(summary_table_cough_abs_incdt)
summary_table_processed_cough_no_abs_prevl = data_process(summary_table_cough_no_abs_prevl)
summary_table_processed_cough_abs_prevl = data_process(summary_table_cough_abs_prevl)
summary_table_processed_cough_cold_no_abs_incdt = data_process(summary_table_cough_cold_no_abs_incdt)
summary_table_processed_cough_cold_abs_incdt = data_process(summary_table_cough_cold_abs_incdt)
summary_table_processed_cough_cold_no_abs_prevl = data_process(summary_table_cough_cold_no_abs_prevl)
summary_table_processed_cough_cold_abs_prevl = data_process(summary_table_cough_cold_abs_prevl)
summary_table_processed_throat_no_abs_incdt = data_process(summary_table_throat_no_abs_incdt)
summary_table_processed_throat_abs_incdt = data_process(summary_table_throat_abs_incdt)
summary_table_processed_throat_no_abs_prevl = data_process(summary_table_throat_no_abs_prevl)
summary_table_processed_throat_abs_prevl = data_process(summary_table_throat_abs_prevl)

In [1]:
d = {'variable': summary_table_processed_lrti_no_abs_incdt['variable'],
     
     'lrti_no_abs_incdt': summary_table_processed_lrti_no_abs_incdt['count_mean'],
     'lrti_abs_incdt': summary_table_processed_lrti_abs_incdt['count_mean'],
     'lrti_no_abs_prevl': summary_table_processed_lrti_no_abs_prevl['count_mean'],
     'lrti_abs_prevl': summary_table_processed_lrti_abs_prevl['count_mean'],

     'urti_combined_no_abs_incdt': summary_table_processed_urti_combined_no_abs_incdt['count_mean'],
     'urti_combined_abs_incdt': summary_table_processed_urti_combined_abs_incdt['count_mean'],
     'urti_combined_no_abs_prevl': summary_table_processed_urti_combined_no_abs_prevl['count_mean'],
     'urti_combined_abs_prevl': summary_table_processed_urti_combined_abs_prevl['count_mean'],

    'uti_no_abs_incdt': summary_table_processed_uti_no_abs_incdt['count_mean'],
    'uti_abs_incdt': summary_table_processed_uti_abs_incdt['count_mean'],
    'uti_no_abs_prevl': summary_table_processed_uti_no_abs_prevl['count_mean'],
    'uti_abs_prevl': summary_table_processed_uti_abs_prevl['count_mean'],

    'sinusitis_no_abs_incdt': summary_table_processed_sinusitis_no_abs_incdt['count_mean'],
    'sinusitis_abs_incdt': summary_table_processed_sinusitis_abs_incdt['count_mean'],
    'sinusitis_no_abs_prevl': summary_table_processed_sinusitis_no_abs_prevl['count_mean'],
    'sinusitis_abs_prevl': summary_table_processed_sinusitis_abs_prevl['count_mean'],

    'ot_externa_no_abs_incdt': summary_table_processed_ot_externa_no_abs_incdt['count_mean'],
    'ot_externa_abs_incdt': summary_table_processed_ot_externa_abs_incdt['count_mean'],
    'ot_externa_no_abs_prevl': summary_table_processed_ot_externa_no_abs_prevl['count_mean'],
    'ot_externa_abs_prevl': summary_table_processed_ot_externa_abs_prevl['count_mean'],

    'otmedia_no_abs_incdt': summary_table_processed_otmedia_no_abs_incdt['count_mean'],
    'otmedia_abs_incdt': summary_table_processed_otmedia_abs_incdt['count_mean'],
    'otmedia_no_abs_prevl': summary_table_processed_otmedia_no_abs_prevl['count_mean'],
    'otmedia_abs_prevl': summary_table_processed_otmedia_abs_prevl['count_mean'],

    'urti_no_abs_incdt': summary_table_processed_urti_no_abs_incdt['count_mean'],
    'urti_abs_incdt': summary_table_processed_urti_abs_incdt['count_mean'],
    'urti_no_abs_prevl': summary_table_processed_urti_no_abs_prevl['count_mean'],
    'urti_abs_prevl': summary_table_processed_urti_abs_prevl['count_mean'],

    'cough_no_abs_incdt': summary_table_processed_cough_no_abs_incdt['count_mean'],
    'cough_abs_incdt': summary_table_processed_cough_abs_incdt['count_mean'],
    'cough_no_abs_prevl': summary_table_processed_cough_no_abs_prevl['count_mean'],
    'cough_abs_prevl': summary_table_processed_cough_abs_prevl['count_mean'],

    'cough_cold_no_abs_incdt': summary_table_processed_cough_cold_no_abs_incdt['count_mean'],
    'cough_cold_abs_incdt': summary_table_processed_cough_cold_abs_incdt['count_mean'],
    'cough_cold_no_abs_prevl': summary_table_processed_cough_cold_no_abs_prevl['count_mean'],
    'cough_cold_abs_prevl': summary_table_processed_cough_cold_abs_prevl['count_mean'],

    'throat_no_abs_incdt': summary_table_processed_throat_no_abs_incdt['count_mean'],
    'throat_abs_incdt': summary_table_processed_throat_abs_incdt['count_mean'],
    'throat_no_abs_prevl': summary_table_processed_throat_no_abs_prevl['count_mean'],
    'throat_abs_prevl': summary_table_processed_throat_abs_prevl['count_mean'], 
    }
data = pd.DataFrame(data=d)

data.to_csv(f'../output/aggregate_tables/baseline_table_all_infections.csv', index=False)